In [1]:
# import statements
#!pip install pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import isnull, when, count, col
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import MulticlassMetrics

sc = SparkContext.getOrCreate()
#spark = SparkSession(sc) 
spark = SparkSession.builder.appName("LogisticRegression").getOrCreate()

In [2]:
spark

#### Q1.  Print Spark version  (1 mark)



In [3]:
spark.version

'3.5.5'

#### Q2. Read adult_data into a Spark-dataframe , databricks_datasets path is - 'dbfs:/databricks-datasets/adult/adult.data'  (1 mark)

In [4]:
df = spark.read.format("csv").option("header","false").option("inferschema","True").load("adult.data")

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)



In [7]:
display(df)

DataFrame[_c0: int, _c1: string, _c2: double, _c3: string, _c4: double, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: double, _c11: double, _c12: double, _c13: string, _c14: string]

In [8]:
df.show(5,truncate=False)

+---+-----------------+--------+----------+----+-------------------+------------------+--------------+------+-------+------+----+----+--------------+------+
|_c0|_c1              |_c2     |_c3       |_c4 |_c5                |_c6               |_c7           |_c8   |_c9    |_c10  |_c11|_c12|_c13          |_c14  |
+---+-----------------+--------+----------+----+-------------------+------------------+--------------+------+-------+------+----+----+--------------+------+
|39 | State-gov       |77516.0 | Bachelors|13.0| Never-married     | Adm-clerical     | Not-in-family| White| Male  |2174.0|0.0 |40.0| United-States| <=50K|
|50 | Self-emp-not-inc|83311.0 | Bachelors|13.0| Married-civ-spouse| Exec-managerial  | Husband      | White| Male  |0.0   |0.0 |13.0| United-States| <=50K|
|38 | Private         |215646.0| HS-grad  |9.0 | Divorced          | Handlers-cleaners| Not-in-family| White| Male  |0.0   |0.0 |40.0| United-States| <=50K|
|53 | Private         |234721.0| 11th     |7.0 | Married-c

In [9]:
df.take(5)

[Row(_c0=39, _c1=' State-gov', _c2=77516.0, _c3=' Bachelors', _c4=13.0, _c5=' Never-married', _c6=' Adm-clerical', _c7=' Not-in-family', _c8=' White', _c9=' Male', _c10=2174.0, _c11=0.0, _c12=40.0, _c13=' United-States', _c14=' <=50K'),
 Row(_c0=50, _c1=' Self-emp-not-inc', _c2=83311.0, _c3=' Bachelors', _c4=13.0, _c5=' Married-civ-spouse', _c6=' Exec-managerial', _c7=' Husband', _c8=' White', _c9=' Male', _c10=0.0, _c11=0.0, _c12=13.0, _c13=' United-States', _c14=' <=50K'),
 Row(_c0=38, _c1=' Private', _c2=215646.0, _c3=' HS-grad', _c4=9.0, _c5=' Divorced', _c6=' Handlers-cleaners', _c7=' Not-in-family', _c8=' White', _c9=' Male', _c10=0.0, _c11=0.0, _c12=40.0, _c13=' United-States', _c14=' <=50K'),
 Row(_c0=53, _c1=' Private', _c2=234721.0, _c3=' 11th', _c4=7.0, _c5=' Married-civ-spouse', _c6=' Handlers-cleaners', _c7=' Husband', _c8=' Black', _c9=' Male', _c10=0.0, _c11=0.0, _c12=40.0, _c13=' United-States', _c14=' <=50K'),
 Row(_c0=28, _c1=' Private', _c2=338409.0, _c3=' Bachelors'

In [10]:
df.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14']

#### Q3. Rename columns of spark dataframe same as given below list  ( 1 mark)
column_names = [
     "age",
     "workclass",
     "final_weight",
     "education",
     "education_num",
     "marital_status",
     "occupation",
     "relationship",
     "race",
     "sex",
     "capital_gain",
     "capital_loss",
     "hours_per_week",
     "native_country",
     "income_class"
 ]

In [11]:
column_names = [ "age", "workclass", "final_weight", "education", "education_num", "marital_status",
                "occupation", "relationship", "race", "sex", 
                "capital_gain", "capital_loss", "hours_per_week", "native_country", "income_class" ]

In [15]:
for new_col,old_col in zip(column_names,df.columns):
    df = df.withColumnRenamed(old_col,new_col)

In [16]:
df.columns

['age',
 'workclass',
 'final_weight',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'income_class']

In [17]:
df.show(5)

+---+-----------------+------------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------------+
|age|        workclass|final_weight| education|education_num|     marital_status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income_class|
+---+-----------------+------------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------------+
| 39|        State-gov|     77516.0| Bachelors|         13.0|      Never-married|      Adm-clerical| Not-in-family| White|   Male|      2174.0|         0.0|          40.0| United-States|       <=50K|
| 50| Self-emp-not-inc|     83311.0| Bachelors|         13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|         0.0|         0.0|          13.0| United-States|       <=50K|


#### Q4. Check if spark-dataframe has any columns with missing/null values and drop such columns  (1 mark)

In [18]:
from pyspark.sql.functions import isnull,when,count,col

In [ ]:
SELECT count(CASE WHEN age is null the age end) age,
       

In [28]:
[count(when(isnull(c),c)).alias(c) for c in df.columns]

[Column<'count(CASE WHEN (age IS NULL) THEN age END) AS age'>,
 Column<'count(CASE WHEN (workclass IS NULL) THEN workclass END) AS workclass'>,
 Column<'count(CASE WHEN (final_weight IS NULL) THEN final_weight END) AS final_weight'>,
 Column<'count(CASE WHEN (education IS NULL) THEN education END) AS education'>,
 Column<'count(CASE WHEN (education_num IS NULL) THEN education_num END) AS education_num'>,
 Column<'count(CASE WHEN (marital_status IS NULL) THEN marital_status END) AS marital_status'>,
 Column<'count(CASE WHEN (occupation IS NULL) THEN occupation END) AS occupation'>,
 Column<'count(CASE WHEN (relationship IS NULL) THEN relationship END) AS relationship'>,
 Column<'count(CASE WHEN (race IS NULL) THEN race END) AS race'>,
 Column<'count(CASE WHEN (sex IS NULL) THEN sex END) AS sex'>,
 Column<'count(CASE WHEN (capital_gain IS NULL) THEN capital_gain END) AS capital_gain'>,
 Column<'count(CASE WHEN (capital_loss IS NULL) THEN capital_loss END) AS capital_loss'>,
 Column<'coun

In [20]:
df.select([count(when(isnull(c),c)).alias(c) for c in df.columns]).show()

+---+---------+------------+---------+-------------+--------------+----------+------------+----+---+------------+------------+--------------+--------------+------------+
|age|workclass|final_weight|education|education_num|marital_status|occupation|relationship|race|sex|capital_gain|capital_loss|hours_per_week|native_country|income_class|
+---+---------+------------+---------+-------------+--------------+----------+------------+----+---+------------+------------+--------------+--------------+------------+
|  0|        0|           0|        0|            0|             0|         0|           0|   0|  0|           0|           0|             0|             0|           0|
+---+---------+------------+---------+-------------+--------------+----------+------------+----+---+------------+------------+--------------+--------------+------------+



In [23]:
df.groupBy("marital_status").count().show(truncate=False)

+----------------------+-----+
|marital_status        |count|
+----------------------+-----+
| Widowed              |993  |
| Married-spouse-absent|418  |
| Married-AF-spouse    |23   |
| Married-civ-spouse   |14976|
| Divorced             |4443 |
| Never-married        |10683|
| Separated            |1025 |
+----------------------+-----+



In [27]:
df.count()

32561

In [30]:
df.describe().show()

+-------+------------------+------------+------------------+-------------+-----------------+--------------+-----------------+------------+-------------------+-------+------------------+----------------+------------------+--------------+------------+
|summary|               age|   workclass|      final_weight|    education|    education_num|marital_status|       occupation|relationship|               race|    sex|      capital_gain|    capital_loss|    hours_per_week|native_country|income_class|
+-------+------------------+------------+------------------+-------------+-----------------+--------------+-----------------+------------+-------------------+-------+------------------+----------------+------------------+--------------+------------+
|  count|             32561|       32561|             32561|        32561|            32561|         32561|            32561|       32561|              32561|  32561|             32561|           32561|             32561|         32561|       32561|


#### Q5. Show the datatypes(schema) of each Spark-dataframe columns without using any for loop (1 mark)

In [31]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- final_weight: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- income_class: string (nullable = true)



#### Q6. Convert all string columns of Spark-dataframe into indexes using  StringIndexer transformer.( reomove  string columns from data-frame and name the target indexed column as label). (3 marks)

In [32]:
indexers = StringIndexer(inputCols = ['workclass','education',"marital_status","occupation",
                                     "relationship","race","sex" , "native_country", 'income_class'],
                         
                         outputCols =['workclass_indexed', 'education_indexed', 
                                     "marital_status_indexed","occupation_indexed","relationship_indexed",
                                     "race_indexed","sex_indexed" ,'native_country_indexed' ,"label"]).fit(df)
df = indexers.transform(df)

In [34]:
df.show(5)

+---+-----------------+------------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------------+-----------------+-----------------+----------------------+------------------+--------------------+------------+-----------+----------------------+-----+
|age|        workclass|final_weight| education|education_num|     marital_status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|income_class|workclass_indexed|education_indexed|marital_status_indexed|occupation_indexed|relationship_indexed|race_indexed|sex_indexed|native_country_indexed|label|
+---+-----------------+------------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------------+-----------------+-----------------+----------------------+------------------+-----------------

In [35]:
cols = ('workclass','education',"marital_status","occupation","relationship","race","sex" , "native_country", 'income_class')
df =df.drop(*cols)
df.show(5, False)

+---+------------+-------------+------------+------------+--------------+-----------------+-----------------+----------------------+------------------+--------------------+------------+-----------+----------------------+-----+
|age|final_weight|education_num|capital_gain|capital_loss|hours_per_week|workclass_indexed|education_indexed|marital_status_indexed|occupation_indexed|relationship_indexed|race_indexed|sex_indexed|native_country_indexed|label|
+---+------------+-------------+------------+------------+--------------+-----------------+-----------------+----------------------+------------------+--------------------+------------+-----------+----------------------+-----+
|39 |77516.0     |13.0         |2174.0      |0.0         |40.0          |4.0              |2.0              |1.0                   |3.0               |1.0                 |0.0         |0.0        |0.0                   |0.0  |
|50 |83311.0     |13.0         |0.0         |0.0         |13.0          |1.0              |2

In [36]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- final_weight: double (nullable = true)
 |-- education_num: double (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- workclass_indexed: double (nullable = false)
 |-- education_indexed: double (nullable = false)
 |-- marital_status_indexed: double (nullable = false)
 |-- occupation_indexed: double (nullable = false)
 |-- relationship_indexed: double (nullable = false)
 |-- race_indexed: double (nullable = false)
 |-- sex_indexed: double (nullable = false)
 |-- native_country_indexed: double (nullable = false)
 |-- label: double (nullable = false)



#### Q7. Using vectorAssembler combines all columns (except  label)  of Sparkdataframe into single column named features (3 marks)

In [37]:
features_col = ['age',
 'final_weight',
 'education_num',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'education_indexed',
 'race_indexed',
 'sex_indexed',
 'workclass_indexed',
 'occupation_indexed',
 'marital_status_indexed',
 'relationship_indexed',
 'native_country_indexed']

assembler = VectorAssembler(inputCols= features_col, outputCol= "features")
df_assembled = assembler.transform(df)
df_assembled.show(5, False)


+---+------------+-------------+------------+------------+--------------+-----------------+-----------------+----------------------+------------------+--------------------+------------+-----------+----------------------+-----+-------------------------------------------------------------------+
|age|final_weight|education_num|capital_gain|capital_loss|hours_per_week|workclass_indexed|education_indexed|marital_status_indexed|occupation_indexed|relationship_indexed|race_indexed|sex_indexed|native_country_indexed|label|features                                                           |
+---+------------+-------------+------------+------------+--------------+-----------------+-----------------+----------------------+------------------+--------------------+------------+-----------+----------------------+-----+-------------------------------------------------------------------+
|39 |77516.0     |13.0         |2174.0      |0.0         |40.0          |4.0              |2.0              |1.0   

#### Q8.  Split the vectorised spark dataframe into training and test sets  (with one third being held for  testing) ( 3 marks)

In [39]:
df_train,df_test = df_assembled.randomSplit([0.8,0.2],seed=123)

In [40]:
df_train.count()

25997

In [41]:
df_test.count()

6564

#### Q9. Train default logistic regression  model with   'featuresCol' as  features and  features as ' label'  (3 marks)

In [42]:
lr = LogisticRegression(featuresCol="features",labelCol="label")
lr_model = lr.fit(df_train)

In [44]:
lr_model.coefficients

DenseVector([0.0416, 0.0, 0.3151, 0.0003, 0.0006, 0.0339, -0.0106, -0.1576, -0.4481, 0.0256, -0.0678, -1.0492, 0.0546, -0.0063])

In [45]:
lr_model.coefficientMatrix

DenseMatrix(1, 14, [0.0416, 0.0, 0.3151, 0.0003, 0.0006, 0.0339, -0.0106, -0.1576, -0.4481, 0.0256, -0.0678, -1.0492, 0.0546, -0.0063], 1)

In [46]:
lr_model.intercept

-7.041949498330972

#### Q10. Find accuracy of   logistic regression model  on test set ( 3 marks)

In [48]:
#This LogisticRegressionModel can be used as a transformer to 
#perform prediction on the testing data
predictonDF = lr_model.transform(df_test)
evaluator = BinaryClassificationEvaluator()

In [49]:
evaluator.evaluate(predictonDF)

0.8816411591987768

In [50]:
evaluator = BinaryClassificationEvaluator(metricName="areaUnderPR")

In [51]:
evaluator.evaluate(predictonDF)

0.7301761997367064

In [52]:
accuracy = predictonDF.filter(predictonDF.label == predictonDF.prediction).count()/float(predictonDF.count())
print("Accuracy = ", accuracy)
 

Accuracy =  0.8362279098110909


In [53]:
selected_predictonDF = predictonDF.select("label", "prediction")
df = selected_predictonDF.toPandas()

In [54]:
df.head()

,label,prediction
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [55]:
from sklearn.metrics import confusion_matrix

In [56]:
confusion_matrix(df["label"],df["prediction"])

array([[4678,  282],
       [ 793,  811]], dtype=int64)